In [1]:
#this is a combination of LSTM		BIRNN		LSTM-LSTM		BiRNN-BIRNN		BiRNN-LSTM
#choose 3 best model from 5 models. 
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

seed_value= 1234
# 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set the `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set the `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)

from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import pandas as pd
import numpy as np
from numpy import array
from numpy import asarray
import numpy as np
import sklearn
from matplotlib import *
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings("ignore")

#Setting GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=4096)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)
    
import warnings
warnings.filterwarnings("ignore")


##End setting for random state.
from sklearn.metrics import f1_score
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import warnings
warnings.filterwarnings("ignore")

def Format_output(y_train_pred):
    res= np.array([])
    for x in y_train_pred:
        if x>=0.5:
            res= np.append(res, 1)
        else: 
            res= np.append(res, 0)
    res= np.reshape(res,(res.shape[0],1))
    return res

#1. LSTM model
def My_LSTM(nodes, X_train, y_train, X_Val, y_Val,model_name):
    import os
    os.environ['PYTHONHASHSEED']=str(seed_value)
    
    import random
    random.seed(seed_value)
    
    import numpy as np
    np.random.seed(seed_value)
    
    tf.random.set_seed(seed_value)
    
    
    es = tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss', patience=50,
                    restore_best_weights =True,verbose=0)
    mc = tf.keras.callbacks.ModelCheckpoint(
                             filepath=model_name,
                             monitor='val_loss', 
                             save_best_only=True)
    callback = [es,mc]
    opt = tf.keras.optimizers.Adam(learning_rate=0.05)

    my_model = tf.keras.Sequential()
    my_model.add(tf.keras.layers.LSTM(units = nodes,input_shape = (X_train.shape[1],X_train.shape[2])))
    my_model.add(tf.keras.layers.Dropout(0.2))
    my_model.add(tf.keras.layers.Dense(20))
    my_model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    my_model.compile(loss = 'mse',
     optimizer = opt, 
     metrics = [tf.keras.metrics.BinaryAccuracy()])

    #my_model.summary()
    my_model.fit(X_train,y_train,
                    validation_data=(X_Val, y_Val),
                    batch_size = 500,epochs = 300,
                    callbacks=callback, verbose=0)
    return my_model

#2. BiRNN model
def My_BiRNN(nodes, X_train, y_train, X_Val, y_Val,model_name):
    import os
    os.environ['PYTHONHASHSEED']=str(seed_value)
    
    import random
    random.seed(seed_value)
    
    import numpy as np
    np.random.seed(seed_value)
    
    tf.random.set_seed(seed_value)
    
    
    es = tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss', patience=50,
                    restore_best_weights =True,verbose=0)
    mc = tf.keras.callbacks.ModelCheckpoint(
                             filepath=model_name,
                             monitor='val_loss', 
                             save_best_only=True)
    callback = [es,mc]
    opt = tf.keras.optimizers.Adam(learning_rate=0.05)

    my_model = tf.keras.Sequential()
    my_model.add(tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(
            units = nodes,
            input_shape = (X_train.shape[1],X_train.shape[2])))
        )
    my_model.add(tf.keras.layers.Dropout(0.2))
    my_model.add(tf.keras.layers.Dense(20))
    my_model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    my_model.compile(loss = 'mse',
     optimizer = opt, 
     metrics = [tf.keras.metrics.BinaryAccuracy()])

    #my_model.summary()
    my_model.fit(X_train,y_train,
                    validation_data=(X_Val, y_Val),
                    batch_size = 500,epochs = 300,
                    callbacks=callback, verbose=0)
    return my_model

#3. LSTM-LSTM model
def My_LSTM_LSTM(nodes, X_train, y_train, X_Val, y_Val,model_name):
    import os
    os.environ['PYTHONHASHSEED']=str(seed_value)
    
    import random
    random.seed(seed_value)
    
    import numpy as np
    np.random.seed(seed_value)
    
    tf.random.set_seed(seed_value)
    
    
    es = tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss', patience=50,
                    restore_best_weights =True,verbose=0)
    mc = tf.keras.callbacks.ModelCheckpoint(
                             filepath=model_name,
                             monitor='val_loss', 
                             save_best_only=True)
    callback = [es,mc]
    opt = tf.keras.optimizers.Adam(learning_rate=0.05)

    my_model = tf.keras.Sequential()
    my_model.add(tf.keras.layers.LSTM(units = nodes,input_shape = (X_train.shape[1],X_train.shape[2]),return_sequences=True))
    my_model.add(tf.keras.layers.Dropout(0.2))
    my_model.add(tf.keras.layers.LSTM(units = 50,input_shape = (X_train.shape[1],X_train.shape[2])))
    my_model.add(tf.keras.layers.Dense(10))
    my_model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    my_model.compile(loss = 'mse',
     optimizer = opt, 
     metrics = [tf.keras.metrics.BinaryAccuracy()])

    #my_model.summary()
    my_model.fit(X_train,y_train,
                    validation_data=(X_Val, y_Val),
                    batch_size = 500,epochs = 300,
                    callbacks=callback, verbose=0)
    return my_model

#4. BiRNN-BiRNNmodel
def My_BiRNN_BiRNN(nodes, X_train, y_train, X_Val, y_Val,model_name):
    import os
    os.environ['PYTHONHASHSEED']=str(seed_value)
    
    import random
    random.seed(seed_value)
    
    import numpy as np
    np.random.seed(seed_value)
    
    tf.random.set_seed(seed_value)
    
    
    es = tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss', patience=50,
                    restore_best_weights =True,verbose=0)
    mc = tf.keras.callbacks.ModelCheckpoint(
                             filepath=model_name,
                             monitor='val_loss', 
                             save_best_only=True)
    callback = [es,mc]
    opt = tf.keras.optimizers.Adam(learning_rate=0.05)

    my_model = tf.keras.Sequential()
    my_model.add(tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(
            units = nodes,
            input_shape = (X_train.shape[1],X_train.shape[2]),return_sequences=True))
        )
    my_model.add(tf.keras.layers.Dropout(0.2))
    my_model.add(tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(
            units = 50,
            input_shape = (X_train.shape[1],X_train.shape[2])))
        )
    my_model.add(tf.keras.layers.Dense(10))
    my_model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    my_model.compile(loss = 'mse',
     optimizer = opt, 
     metrics = [tf.keras.metrics.BinaryAccuracy()])

    #my_model.summary()
    my_model.fit(X_train,y_train,
                    validation_data=(X_Val, y_Val),
                    batch_size = 500,epochs = 300,
                    callbacks=callback, verbose=0)
    return my_model

#5. BiRNN-LSTM
def My_BiRNN_LSTM(nodes, X_train, y_train, X_Val, y_Val,model_name):
    import os
    os.environ['PYTHONHASHSEED']=str(seed_value)
    
    import random
    random.seed(seed_value)
    
    import numpy as np
    np.random.seed(seed_value)
    
    tf.random.set_seed(seed_value)
    
    
    es = tf.keras.callbacks.EarlyStopping(
                    monitor='val_loss', patience=50,
                    restore_best_weights =True,verbose=0)
    mc = tf.keras.callbacks.ModelCheckpoint(
                             filepath=model_name,
                             monitor='val_loss', 
                             save_best_only=True)
    callback = [es,mc]
    opt = tf.keras.optimizers.Adam(learning_rate=0.05)

    my_model = tf.keras.Sequential()
    my_model.add(tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(
            units = nodes,
            input_shape = (X_train.shape[1],X_train.shape[2]),return_sequences=True))
        )
    my_model.add(tf.keras.layers.Dropout(0.2))
    my_model.add(tf.keras.layers.LSTM(units = 50,input_shape = (X_train.shape[1],X_train.shape[2])))
    my_model.add(tf.keras.layers.Dense(10))
    my_model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    my_model.compile(loss = 'mse',
     optimizer = opt, 
     metrics = [tf.keras.metrics.BinaryAccuracy()])

    #my_model.summary()
    my_model.fit(X_train,y_train,
                    validation_data=(X_Val, y_Val),
                    batch_size = 500,epochs = 300,
                    callbacks=callback, verbose=0)
    return my_model

class My_Model():
    def __init__(self,):
        
        import os
        os.environ['PYTHONHASHSEED']=str(seed_value)

        import random
        random.seed(seed_value)

        import numpy as np
        np.random.seed(seed_value)

        tf.random.set_seed(seed_value)

        self.Models = []
        
        #train accuracy.
        self.Precisions_train = []
        self.Recalls_train   = []
        self.F_scores_train  = []
        self.AUCs_train      = []
        #Test accuracy.
        self.Precisions_test = []
        self.Recalls_test   = []
        self.F_scores_test  = []
        self.AUCs_test      = []
        
    
    def run_main(self,i,x_train, y_train, x_test, y_test):
        List_Nodes= [16,32,64,128,256]
        temp_auc = -1
        temp_Model_LSTM = None
        for nodes in List_Nodes:
            if (i==1):
                Model = My_LSTM(nodes, x_train, y_train, x_test, y_test,'temp.h5')
                print('LSTM model with ',nodes,' nodes stop after ',len(Model.history.history['loss']), ' epochs')
            
            elif(i==2):
                Model = My_BiRNN(nodes, x_train, y_train, x_test, y_test,'temp.h5')
                print('BiRNN model with ',nodes,' nodes stop after ',len(Model.history.history['loss']), ' epochs')               
            
            elif(i==3):
                
                Model = My_LSTM_LSTM(nodes, x_train, y_train, x_test, y_test,'temp.h5')
                print('LSTM_LSTM model with ',nodes,' nodes stop after ',len(Model.history.history['loss']), ' epochs')               
                
            elif(i==4):

                Model = My_BiRNN_BiRNN(nodes, x_train, y_train, x_test, y_test,'temp.h5')
                print('BiRNN_BiRNN model with ',nodes,' nodes stop after ',len(Model.history.history['loss']), ' epochs')               
                
            elif(i==5):

                Model = My_BiRNN_LSTM(nodes, x_train, y_train, x_test, y_test,'temp.h5')
                print('BiRNN_LSTM model with ',nodes,' nodes stop after ',len(Model.history.history['loss']), ' epochs')               
        
                pass
            else:
                print('i parameter is from 1 to 5.')
            y_pred = Format_output(Model.predict(x_test))
            fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
            AUC_test = metrics.auc(fpr, tpr)
            print('AUC = ',AUC_test)
            
            if(AUC_test > temp_auc):
                print('Current status: ',nodes,' nodes')
                temp_Model_LSTM = Model
                temp_auc = AUC_test
        #compute metrics for lstm model
        #train
        y_train_pred = Format_output(temp_Model_LSTM.predict(x_train))
        precision_train,recall_train,f1Score_train,_ = precision_recall_fscore_support(y_train, y_train_pred,average='binary')
        fpr, tpr, thresholds = metrics.roc_curve(y_train, y_train_pred)
        AUC_train = metrics.auc(fpr, tpr)
        self.Precisions_train.append(precision_train)
        self.Recalls_train.append(recall_train)
        self.F_scores_train.append(f1Score_train)
        self.AUCs_train.append(AUC_train)
        #test
        y_pred = Format_output(temp_Model_LSTM.predict(x_test))
        precision_test,recall_test,f1Score_test,_ = precision_recall_fscore_support(y_test, y_pred,average='binary')
        fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
        AUC_test = metrics.auc(fpr, tpr)
        self.Precisions_test.append(precision_test)
        self.Recalls_test.append(recall_test)
        self.F_scores_test.append(f1Score_test)
        self.AUCs_test.append(AUC_test)
        
        import os
        os.remove('temp.h5')
        self.Models.append(temp_Model_LSTM)
        
    #training all models.
    def My_fit(self,x_train, y_train, x_test, y_test):       
        #LSTM
        print('\n1. LSTM Model: ')
        self.run_main(1,x_train, y_train, x_test, y_test)
        #BiRNN
        print('\n2. BiRNN Model: ')
        self.run_main(2,x_train, y_train, x_test, y_test)
        #LSTM_LSTM
        print('\n3. LSTM_LSTM Model: ')
        self.run_main(3,x_train, y_train, x_test, y_test)
        #BiRNN_BiRNN
        print('\n4. BiRNN_BiRNN Model: ')
        self.run_main(4,x_train, y_train, x_test, y_test)
        #BiRNN-LSTM
        print('\n5. BiRNN-LSTM Model: ')
        self.run_main(5,x_train, y_train, x_test, y_test)
        
        #Chose top 3 models to predict in the future.
        #get top 3 models
        L1= np.array(self.AUCs_train)
        index = numpy.argsort(L1)[2:]
        Models2 = []
        for i in index:
            Models2.append(self.Models[i])
            
        self.Models = Models2 
        
        print('Done Training')

    #Predict all models.
    def predict(self,x_data):     

        #compute the average for train and test. 
        y_preds = []
        for model in self.Models:
            #predict and compute the average values. 
            y_pred = model.predict(x_data)
            y_preds.append(y_pred)
        
        #Format_output(
        #do it here
        Results = np.array([])

        for i in range (0,len(y_preds[0])):
            sum = 0
            for j in range(len(y_preds)):
                sum += (y_preds[j][i])
            if(sum/3.0 >= 0.5):
                Results = np.append(Results,1)
            else:
                Results = np.append(Results,0)
        
        Results= np.reshape(Results,(Results.shape[0],1))
        return Results
    
    def save(self, Filename="EnsembleDL"):
        i=1
        for model in self.Models:
            Name = Filename+'_model'+str(i)+'.h5'
            model.save(Name)
            i+=1
            print('Write submodel ',1,' to ', Name)
        print('Save Ensemble DL Done...')


def Load_EnsembleDL(Filenames): #Filenames is include all submodel files
    EnsembleDL = My_Model()
    i=1
    Models = []
    for filename in Filenames:
        model = keras.models.load_model(filename)
        EnsembleDL.Models.append(model)
        print('Load submodel ',i,' from ', filename)
        i+=1
    print('Load Ensemble DL Done...')
    return EnsembleDL
                      
def Run(Train_Path,Test_Path,model_name='None.h5'):

    #def run_GBM(Train_Path, Test_Path):
    Train = pd.read_csv(Train_Path)
    Test  = pd.read_csv(Test_Path)
    Train = Train.dropna()
    Test  = Test.dropna()

    New_data = [Train,Test]
    New_data = pd.concat(New_data)

    scaler = MinMaxScaler()
    New_data = scaler.fit_transform(New_data)

    Train =New_data[0:Train.shape[0],:]
    Test = New_data[(Train.shape[0]):,:]

    x_train = Train[:,0:-1]
    y_train = Train[:,-1]
    
    x_test = Test[:,0:-1]
    y_test = Test[:,-1]

    print( 'size before appling reshape: ',x_train.shape,y_train.shape,x_test.shape, y_test.shape)
    #minmax scaler for training and testing
    
    x_train = np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))
    y_train = np.reshape(y_train,(y_train.shape[0],1))

    x_test = np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))
    y_test = np.reshape(y_test,(y_test.shape[0],1))
    print( 'size after appling reshape: ',x_train.shape,y_train.shape,x_test.shape, y_test.shape)


    #create model.
    Ensemble_DL = My_Model()
    Ensemble_DL.My_fit(x_train, y_train, x_test, y_test)
    
    print(Ensemble_DL.Precisions_train)
    print(Ensemble_DL.Recalls_train)
    print(Ensemble_DL.F_scores_train)
    print(Ensemble_DL.AUCs_train)
    print(Ensemble_DL.Precisions_test)
    print(Ensemble_DL.Recalls_test)
    print(Ensemble_DL.F_scores_test)
    print(Ensemble_DL.AUCs_test)

    #Ensemble results.
    y_train_pred = Format_output(Ensemble_DL.My_Predict(x_train))
    precision_train,recall_train,f1Score_train,_ = precision_recall_fscore_support(y_train, y_train_pred,average='binary')
    fpr, tpr, thresholds = metrics.roc_curve(y_train, y_train_pred)
    AUC_train = metrics.auc(fpr, tpr)
    #test
    y_pred = Format_output(Ensemble_DL.My_Predict(x_test))
    precision_test,recall_test,f1Score_test,_ = precision_recall_fscore_support(y_test, y_pred,average='binary')
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    AUC_test = metrics.auc(fpr, tpr)
    
    print('Ensemble Results: ')
    print("Train: ")
    print("precision: \t",precision_train)
    print("recall:    \t",recall_train)
    print("F score:   \t",f1Score_train)
    print("AUC score: \t",AUC_train)

    print("Test: ")
    print("precision: \t",precision_test)
    print("recall:    \t",recall_test)
    print("F score:   \t",f1Score_test)
    print("AUC score: \t",AUC_test)
        
    
    print('Done Traning...')
    return Ensemble_DL

def Test_Loading(Train_Path,Test_Path,Model_loaded):
    #def run_GBM(Train_Path, Test_Path):
    Train = pd.read_csv(Train_Path)
    Test  = pd.read_csv(Test_Path)
    Train = Train.dropna()
    Test  = Test.dropna()

    New_data = [Train,Test]
    New_data = pd.concat(New_data)

    scaler = MinMaxScaler()
    New_data = scaler.fit_transform(New_data)

    Train =New_data[0:Train.shape[0],:]
    Test = New_data[(Train.shape[0]):,:]

    x_train = Train[:,0:-1]
    y_train = Train[:,-1]
    
    x_test = Test[:,0:-1]
    y_test = Test[:,-1]

    print( 'size before appling reshape: ',x_train.shape,y_train.shape,x_test.shape, y_test.shape)
    #minmax scaler for training and testing
    
    x_train = np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1))
    y_train = np.reshape(y_train,(y_train.shape[0],1))

    x_test = np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))
    y_test = np.reshape(y_test,(y_test.shape[0],1))
    print( 'size after appling reshape: ',x_train.shape,y_train.shape,x_test.shape, y_test.shape)


    #create model.
    Ensemble_DL = Model_loaded
    
    #Ensemble results.
    y_train_pred = Format_output(Ensemble_DL.My_Predict(x_train))
    precision_train,recall_train,f1Score_train,_ = precision_recall_fscore_support(y_train, y_train_pred,average='binary')
    fpr, tpr, thresholds = metrics.roc_curve(y_train, y_train_pred)
    AUC_train = metrics.auc(fpr, tpr)
    #test
    y_pred = Format_output(Ensemble_DL.My_Predict(x_test))
    precision_test,recall_test,f1Score_test,_ = precision_recall_fscore_support(y_test, y_pred,average='binary')
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    AUC_test = metrics.auc(fpr, tpr)
    
    print('Ensemble Results: ')
    print("Train: ")
    print("precision: \t",precision_train)
    print("recall:    \t",recall_train)
    print("F score:   \t",f1Score_train)
    print("AUC score: \t",AUC_train)

    print("Test: ")
    print("precision: \t",precision_test)
    print("recall:    \t",recall_test)
    print("F score:   \t",f1Score_test)
    print("AUC score: \t",AUC_test)
        
    print('Done')

1 Physical GPUs, 1 Logical GPUs


In [2]:
data_path= 'C:\\data\\PeakConcentration-Idea2-2-Clasification\\FullDay_Peak_Values\\'
for i in range(0,5):
    print('\n',i+1)
    Model = Run(data_path+'NO2_Train.csv',data_path+'NO2_Test.csv')
    Model.save(Filename='Saved_Models_top3_2\\'+str(i+1)+"_EnsembleDL_NO2")
    
    #test the saved models.
    #print('Results from load models')
    #Filenames = []
    #for j in range(0,5):
    #    Name = str(i+1)+'_EnsembleDL_NO2'+'_model'+str(j+1)+'.h5'
    #    Filenames.append(Name)

    #Model_loaded = My_Model() #create
    #Model_loaded.Models = Load_EnsembleDL(Filenames)
    #Test_Loading(data_path+'NO2_Train.csv',data_path+'NO2_Test.csv',Model_loaded)



 1
size before appling reshape:  (2900, 11) (2900,) (730, 11) (730,)
size after appling reshape:  (2900, 11, 1) (2900, 1) (730, 11, 1) (730, 1)

1. LSTM Model: 
LSTM model with  16  nodes stop after  114  epochs
AUC =  0.7479452054794521
Current status:  16  nodes
LSTM model with  32  nodes stop after  115  epochs
AUC =  0.7356164383561644
LSTM model with  64  nodes stop after  132  epochs
AUC =  0.7643835616438357
Current status:  64  nodes
LSTM model with  128  nodes stop after  138  epochs
AUC =  0.7479452054794521
LSTM model with  256  nodes stop after  51  epochs
AUC =  0.5

2. BiRNN Model: 
BiRNN model with  16  nodes stop after  173  epochs
AUC =  0.7424657534246575
Current status:  16  nodes
BiRNN model with  32  nodes stop after  123  epochs
AUC =  0.7410958904109589
BiRNN model with  64  nodes stop after  98  epochs
AUC =  0.7547945205479453
Current status:  64  nodes
BiRNN model with  128  nodes stop after  53  epochs
AUC =  0.5
BiRNN model with  256  nodes stop after  51  

AUC =  0.747945205479452
Current status:  16  nodes
BiRNN model with  32  nodes stop after  95  epochs
AUC =  0.7424657534246574
BiRNN model with  64  nodes stop after  102  epochs
AUC =  0.7602739726027397
Current status:  64  nodes
BiRNN model with  128  nodes stop after  53  epochs
AUC =  0.5
BiRNN model with  256  nodes stop after  51  epochs
AUC =  0.5

3. LSTM_LSTM Model: 
LSTM_LSTM model with  16  nodes stop after  109  epochs
AUC =  0.7410958904109589
Current status:  16  nodes
LSTM_LSTM model with  32  nodes stop after  86  epochs
AUC =  0.7452054794520547
Current status:  32  nodes
LSTM_LSTM model with  64  nodes stop after  125  epochs
AUC =  0.7301369863013698
LSTM_LSTM model with  128  nodes stop after  132  epochs
AUC =  0.5095890410958904
LSTM_LSTM model with  256  nodes stop after  86  epochs
AUC =  0.5

4. BiRNN_BiRNN Model: 
BiRNN_BiRNN model with  16  nodes stop after  179  epochs
AUC =  0.7493150684931508
Current status:  16  nodes
BiRNN_BiRNN model with  32  nodes 

AUC =  0.7383561643835617
LSTM_LSTM model with  128  nodes stop after  132  epochs
AUC =  0.5095890410958904
LSTM_LSTM model with  256  nodes stop after  86  epochs
AUC =  0.5

4. BiRNN_BiRNN Model: 
BiRNN_BiRNN model with  16  nodes stop after  103  epochs
AUC =  0.7465753424657535
Current status:  16  nodes
BiRNN_BiRNN model with  32  nodes stop after  97  epochs
AUC =  0.7315068493150685
BiRNN_BiRNN model with  64  nodes stop after  160  epochs
AUC =  0.7657534246575343
Current status:  64  nodes
BiRNN_BiRNN model with  128  nodes stop after  300  epochs
AUC =  0.7301369863013698
BiRNN_BiRNN model with  256  nodes stop after  84  epochs
AUC =  0.5

5. BiRNN-LSTM Model: 
BiRNN_LSTM model with  16  nodes stop after  87  epochs
AUC =  0.736986301369863
Current status:  16  nodes
BiRNN_LSTM model with  32  nodes stop after  96  epochs
AUC =  0.7493150684931507
Current status:  32  nodes
BiRNN_LSTM model with  64  nodes stop after  117  epochs
AUC =  0.741095890410959
BiRNN_LSTM model wi

In [3]:
data_path= 'C:\\data\\PeakConcentration-Idea2-2-Clasification\\FullDay_Peak_Values\\'
for i in range(0,5):
    print('\n',i+1)
    Model = Run(data_path+'CO_Train.csv',data_path+'CO_Test.csv')
    Model.save(Filename='Saved_Models_top3_2\\'+str(i+1)+"_EnsembleDL_CO")



 1
size before appling reshape:  (2906, 11) (2906,) (730, 11) (730,)
size after appling reshape:  (2906, 11, 1) (2906, 1) (730, 11, 1) (730, 1)

1. LSTM Model: 
LSTM model with  16  nodes stop after  138  epochs
AUC =  0.7863013698630138
Current status:  16  nodes
LSTM model with  32  nodes stop after  153  epochs
AUC =  0.7917808219178082
Current status:  32  nodes
LSTM model with  64  nodes stop after  115  epochs
AUC =  0.7712328767123288
LSTM model with  128  nodes stop after  120  epochs
AUC =  0.773972602739726
LSTM model with  256  nodes stop after  54  epochs
AUC =  0.5

2. BiRNN Model: 
BiRNN model with  16  nodes stop after  176  epochs
AUC =  0.7821917808219178
Current status:  16  nodes
BiRNN model with  32  nodes stop after  114  epochs
AUC =  0.7821917808219179
Current status:  32  nodes
BiRNN model with  64  nodes stop after  132  epochs
AUC =  0.7684931506849315
BiRNN model with  128  nodes stop after  51  epochs
AUC =  0.5
BiRNN model with  256  nodes stop after  51  

AUC =  0.5

2. BiRNN Model: 
BiRNN model with  16  nodes stop after  162  epochs
AUC =  0.7684931506849314
Current status:  16  nodes
BiRNN model with  32  nodes stop after  117  epochs
AUC =  0.7780821917808219
Current status:  32  nodes
BiRNN model with  64  nodes stop after  118  epochs
AUC =  0.7698630136986301
BiRNN model with  128  nodes stop after  51  epochs
AUC =  0.5
BiRNN model with  256  nodes stop after  51  epochs
AUC =  0.5

3. LSTM_LSTM Model: 
LSTM_LSTM model with  16  nodes stop after  112  epochs
AUC =  0.7506849315068493
Current status:  16  nodes
LSTM_LSTM model with  32  nodes stop after  110  epochs
AUC =  0.7753424657534246
Current status:  32  nodes
LSTM_LSTM model with  64  nodes stop after  127  epochs
AUC =  0.7767123287671234
Current status:  64  nodes
LSTM_LSTM model with  128  nodes stop after  64  epochs
AUC =  0.5
LSTM_LSTM model with  256  nodes stop after  89  epochs
AUC =  0.5

4. BiRNN_BiRNN Model: 
BiRNN_BiRNN model with  16  nodes stop after  112 

AUC =  0.7945205479452054
Current status:  32  nodes
LSTM_LSTM model with  64  nodes stop after  178  epochs
AUC =  0.7808219178082192
LSTM_LSTM model with  128  nodes stop after  64  epochs
AUC =  0.5
LSTM_LSTM model with  256  nodes stop after  91  epochs
AUC =  0.6219178082191781

4. BiRNN_BiRNN Model: 
BiRNN_BiRNN model with  16  nodes stop after  125  epochs
AUC =  0.7726027397260273
Current status:  16  nodes
BiRNN_BiRNN model with  32  nodes stop after  96  epochs
AUC =  0.7739726027397261
Current status:  32  nodes
BiRNN_BiRNN model with  64  nodes stop after  212  epochs
AUC =  0.7767123287671233
Current status:  64  nodes
BiRNN_BiRNN model with  128  nodes stop after  80  epochs
AUC =  0.6191780821917808
BiRNN_BiRNN model with  256  nodes stop after  95  epochs
AUC =  0.636986301369863

5. BiRNN-LSTM Model: 
BiRNN_LSTM model with  16  nodes stop after  123  epochs
AUC =  0.767123287671233
Current status:  16  nodes
BiRNN_LSTM model with  32  nodes stop after  145  epochs
AUC 

In [4]:
data_path= 'C:\\data\\PeakConcentration-Idea2-2-Clasification\\FullDay_Peak_Values\\'
for i in range(0,5):
    print('\n',i+1)
    Model = Run(data_path+'O3_Train.csv',data_path+'O3_Test.csv')
    Model.save(Filename='Saved_Models_top3_2\\'+str(i+1)+"_EnsembleDL_O3")



 1
size before appling reshape:  (2908, 11) (2908,) (730, 11) (730,)
size after appling reshape:  (2908, 11, 1) (2908, 1) (730, 11, 1) (730, 1)

1. LSTM Model: 
LSTM model with  16  nodes stop after  167  epochs
AUC =  0.8287671232876712
Current status:  16  nodes
LSTM model with  32  nodes stop after  103  epochs
AUC =  0.8383561643835616
Current status:  32  nodes
LSTM model with  64  nodes stop after  130  epochs
AUC =  0.8671232876712329
Current status:  64  nodes
LSTM model with  128  nodes stop after  236  epochs
AUC =  0.8698630136986302
Current status:  128  nodes
LSTM model with  256  nodes stop after  51  epochs
AUC =  0.5

2. BiRNN Model: 
BiRNN model with  16  nodes stop after  117  epochs
AUC =  0.8383561643835618
Current status:  16  nodes
BiRNN model with  32  nodes stop after  111  epochs
AUC =  0.8438356164383561
Current status:  32  nodes
BiRNN model with  64  nodes stop after  167  epochs
AUC =  0.8575342465753425
Current status:  64  nodes
BiRNN model with  128  no

AUC =  0.5945205479452055
LSTM model with  256  nodes stop after  51  epochs
AUC =  0.5

2. BiRNN Model: 
BiRNN model with  16  nodes stop after  120  epochs
AUC =  0.8356164383561644
Current status:  16  nodes
BiRNN model with  32  nodes stop after  106  epochs
AUC =  0.8589041095890411
Current status:  32  nodes
BiRNN model with  64  nodes stop after  163  epochs
AUC =  0.8479452054794521
BiRNN model with  128  nodes stop after  53  epochs
AUC =  0.5
BiRNN model with  256  nodes stop after  51  epochs
AUC =  0.5

3. LSTM_LSTM Model: 
LSTM_LSTM model with  16  nodes stop after  161  epochs
AUC =  0.8561643835616439
Current status:  16  nodes
LSTM_LSTM model with  32  nodes stop after  191  epochs
AUC =  0.8698630136986301
Current status:  32  nodes
LSTM_LSTM model with  64  nodes stop after  109  epochs
AUC =  0.8616438356164383
LSTM_LSTM model with  128  nodes stop after  102  epochs
AUC =  0.5
LSTM_LSTM model with  256  nodes stop after  51  epochs
AUC =  0.6315068493150685

4. BiRN

AUC =  0.5
BiRNN model with  256  nodes stop after  51  epochs
AUC =  0.5

3. LSTM_LSTM Model: 
LSTM_LSTM model with  16  nodes stop after  175  epochs
AUC =  0.8520547945205479
Current status:  16  nodes
LSTM_LSTM model with  32  nodes stop after  157  epochs
AUC =  0.8657534246575342
Current status:  32  nodes
LSTM_LSTM model with  64  nodes stop after  110  epochs
AUC =  0.8671232876712329
Current status:  64  nodes
LSTM_LSTM model with  128  nodes stop after  108  epochs
AUC =  0.5
LSTM_LSTM model with  256  nodes stop after  51  epochs
AUC =  0.6315068493150685

4. BiRNN_BiRNN Model: 
BiRNN_BiRNN model with  16  nodes stop after  98  epochs
AUC =  0.8575342465753424
Current status:  16  nodes
BiRNN_BiRNN model with  32  nodes stop after  122  epochs
AUC =  0.8589041095890411
Current status:  32  nodes
BiRNN_BiRNN model with  64  nodes stop after  300  epochs
AUC =  0.8369863013698631
BiRNN_BiRNN model with  128  nodes stop after  250  epochs
AUC =  0.8589041095890411
BiRNN_BiRNN m

In [5]:
data_path= 'C:\\data\\PeakConcentration-Idea2-2-Clasification\\FullDay_Peak_Values\\'
for i in range(0,5):
    print('\n',i+1)
    Model = Run(data_path+'PM2_5_Train.csv',data_path+'PM2_5_Test.csv')
    Model.save(Filename='Saved_Models_top3_2\\'+str(i+1)+"_EnsembleDL_PM2_5")



 1
size before appling reshape:  (2899, 11) (2899,) (728, 11) (728,)
size after appling reshape:  (2899, 11, 1) (2899, 1) (728, 11, 1) (728, 1)

1. LSTM Model: 
LSTM model with  16  nodes stop after  109  epochs
AUC =  0.6854395604395604
Current status:  16  nodes
LSTM model with  32  nodes stop after  68  epochs
AUC =  0.6689560439560439
LSTM model with  64  nodes stop after  133  epochs
AUC =  0.7101648351648351
Current status:  64  nodes
LSTM model with  128  nodes stop after  52  epochs
AUC =  0.5
LSTM model with  256  nodes stop after  51  epochs
AUC =  0.5

2. BiRNN Model: 
BiRNN model with  16  nodes stop after  92  epochs
AUC =  0.6813186813186815
Current status:  16  nodes
BiRNN model with  32  nodes stop after  79  epochs
AUC =  0.6771978021978022
BiRNN model with  64  nodes stop after  99  epochs
AUC =  0.7046703296703296
Current status:  64  nodes
BiRNN model with  128  nodes stop after  54  epochs
AUC =  0.5
BiRNN model with  256  nodes stop after  52  epochs
AUC =  0.521

AUC =  0.5
LSTM_LSTM model with  256  nodes stop after  80  epochs
AUC =  0.5

4. BiRNN_BiRNN Model: 
BiRNN_BiRNN model with  16  nodes stop after  110  epochs
AUC =  0.7087912087912088
Current status:  16  nodes
BiRNN_BiRNN model with  32  nodes stop after  128  epochs
AUC =  0.7046703296703296
BiRNN_BiRNN model with  64  nodes stop after  240  epochs
AUC =  0.7115384615384616
Current status:  64  nodes
BiRNN_BiRNN model with  128  nodes stop after  106  epochs
AUC =  0.5480769230769231
BiRNN_BiRNN model with  256  nodes stop after  87  epochs
AUC =  0.5

5. BiRNN-LSTM Model: 
BiRNN_LSTM model with  16  nodes stop after  91  epochs
AUC =  0.706043956043956
Current status:  16  nodes
BiRNN_LSTM model with  32  nodes stop after  170  epochs
AUC =  0.7156593406593407
Current status:  32  nodes
BiRNN_LSTM model with  64  nodes stop after  157  epochs
AUC =  0.7019230769230769
BiRNN_LSTM model with  128  nodes stop after  99  epochs
AUC =  0.5618131868131868
BiRNN_LSTM model with  256  nod

In [6]:
data_path= 'C:\\data\\PeakConcentration-Idea2-2-Clasification\\FullDay_Peak_Values\\'
for i in range(0,5):
    print('\n',i+1)
    Model = Run(data_path+'SO2_Train.csv',data_path+'SO2_Test.csv')
    Model.save(Filename='Saved_Models_top3_2\\'+str(i+1)+"_EnsembleDL_SO2")


 1
size before appling reshape:  (2880, 11) (2880,) (728, 11) (728,)
size after appling reshape:  (2880, 11, 1) (2880, 1) (728, 11, 1) (728, 1)

1. LSTM Model: 
LSTM model with  16  nodes stop after  96  epochs
AUC =  0.6964285714285714
Current status:  16  nodes
LSTM model with  32  nodes stop after  93  epochs
AUC =  0.6881868131868132
LSTM model with  64  nodes stop after  141  epochs
AUC =  0.7032967032967032
Current status:  64  nodes
LSTM model with  128  nodes stop after  98  epochs
AUC =  0.6662087912087912
LSTM model with  256  nodes stop after  55  epochs
AUC =  0.5

2. BiRNN Model: 
BiRNN model with  16  nodes stop after  123  epochs
AUC =  0.6758241758241759
Current status:  16  nodes
BiRNN model with  32  nodes stop after  114  epochs
AUC =  0.7142857142857144
Current status:  32  nodes
BiRNN model with  64  nodes stop after  300  epochs
AUC =  0.5851648351648351
BiRNN model with  128  nodes stop after  51  epochs
AUC =  0.5288461538461539
BiRNN model with  256  nodes sto

AUC =  0.6854395604395604
Current status:  16  nodes
BiRNN model with  32  nodes stop after  88  epochs
AUC =  0.6881868131868132
Current status:  32  nodes
BiRNN model with  64  nodes stop after  146  epochs
AUC =  0.5563186813186815
BiRNN model with  128  nodes stop after  51  epochs
AUC =  0.5288461538461539
BiRNN model with  256  nodes stop after  51  epochs
AUC =  0.5

3. LSTM_LSTM Model: 
LSTM_LSTM model with  16  nodes stop after  88  epochs
AUC =  0.6744505494505495
Current status:  16  nodes
LSTM_LSTM model with  32  nodes stop after  106  epochs
AUC =  0.7046703296703297
Current status:  32  nodes
LSTM_LSTM model with  64  nodes stop after  174  epochs
AUC =  0.6923076923076923
LSTM_LSTM model with  128  nodes stop after  53  epochs
AUC =  0.49313186813186816
LSTM_LSTM model with  256  nodes stop after  106  epochs
AUC =  0.5

4. BiRNN_BiRNN Model: 
BiRNN_BiRNN model with  16  nodes stop after  96  epochs
AUC =  0.6881868131868132
Current status:  16  nodes
BiRNN_BiRNN model 

AUC =  0.49313186813186816
LSTM_LSTM model with  256  nodes stop after  106  epochs
AUC =  0.5

4. BiRNN_BiRNN Model: 
BiRNN_BiRNN model with  16  nodes stop after  78  epochs
AUC =  0.695054945054945
Current status:  16  nodes
BiRNN_BiRNN model with  32  nodes stop after  128  epochs
AUC =  0.6909340659340659
BiRNN_BiRNN model with  64  nodes stop after  56  epochs
AUC =  0.614010989010989
BiRNN_BiRNN model with  128  nodes stop after  68  epochs
AUC =  0.5027472527472527
BiRNN_BiRNN model with  256  nodes stop after  73  epochs
AUC =  0.6043956043956045

5. BiRNN-LSTM Model: 
BiRNN_LSTM model with  16  nodes stop after  83  epochs
AUC =  0.6923076923076923
Current status:  16  nodes
BiRNN_LSTM model with  32  nodes stop after  98  epochs
AUC =  0.6717032967032968
BiRNN_LSTM model with  64  nodes stop after  127  epochs
AUC =  0.6991758241758241
Current status:  64  nodes
BiRNN_LSTM model with  128  nodes stop after  96  epochs
AUC =  0.6016483516483517
BiRNN_LSTM model with  256  nod